In [ ]:
import polars as pl
import numpy as np
import scipy.sparse as sparse
from datetime import datetime

# CẤU HÌNH
FILE_PATH = "processed_data/processed_purchase.parquet"
ALPHA = 0.01 # Hệ số suy giảm (Time Decay)

print("--- BƯỚC 1: KHỞI TẠO LAZY FRAME ---")

q = pl.scan_parquet(FILE_PATH)
max_date = q.select(pl.col("date").max()).collect().item()

print(f"Đã load LazyFrame. Ngày mốc (Max Date): {max_date}")

--- BƯỚC 1: KHỞI TẠO LAZY FRAME ---
Đã load LazyFrame. Ngày mốc (Max Date): 2024-12-31


In [2]:
print("--- BƯỚC 2: TÍNH TOÁN TIME DECAY (LAZY) ---")

# Xây dựng các biểu thức tính toán
q_weighted = q.with_columns([
    # 1. Tính số ngày trôi qua: (Max Date - Current Date)
    (max_date - pl.col("date")).dt.total_days().alias("days_ago")
]).with_columns([
    # 2. Tính hệ số suy giảm
    (1.0 / (1.0 + ALPHA * pl.col("days_ago"))).alias("decay_factor")
]).with_columns([
    # 3. Tính trọng số cuối cùng (Weight)
    (pl.col("quantity") * pl.col("decay_factor")).alias("weighted_quantity")
])

--- BƯỚC 2: TÍNH TOÁN TIME DECAY (LAZY) ---


In [3]:
print("--- BƯỚC 3: GOM NHÓM & THỰC THI (COLLECT) ---")

# Group theo User và Item -> Tính tổng trọng số
df_grouped = q_weighted.group_by(["customer_id", "item_id"]).agg(
    pl.col("weighted_quantity").sum()
).collect()

print(f"Dữ liệu sau khi gộp: {df_grouped.shape[0]} dòng")
df_grouped.head(5)

--- BƯỚC 3: GOM NHÓM & THỰC THI (COLLECT) ---
Dữ liệu sau khi gộp: 24700527 dòng


customer_id,item_id,weighted_quantity
i32,str,f64
6001350,"""1439000000009""",1.12782
5846583,"""4396000000003""",0.291545
4460754,"""4837000000004""",0.591716
2203628,"""7228000000021""",2.884615
5472244,"""4783000000001""",1.069519


In [4]:
print("--- BƯỚC 4: MAPPING ID -> INDEX ---")

# 1. Tạo danh sách Unique User và Item (sort để cố định thứ tự)
unique_users = df_grouped.select("customer_id").unique().sort("customer_id")
unique_items = df_grouped.select("item_id").unique().sort("item_id")

# 2. Thêm cột index (row number)
user_map_df = unique_users.with_row_index("user_index")
item_map_df = unique_items.with_row_index("item_index")

# 3. Join bảng map vào bảng chính để lấy index
df_final = df_grouped.join(user_map_df, on="customer_id", how="left") \
                     .join(item_map_df, on="item_id", how="left")

# 4. Tạo Dictionary để tra cứu ngược
user_lookup = dict(zip(user_map_df["user_index"], user_map_df["customer_id"]))
item_lookup = dict(zip(item_map_df["item_index"], item_map_df["item_id"]))

print("Mapping hoàn tất.")
df_final.select(["user_index", "item_index", "weighted_quantity"]).head()

--- BƯỚC 4: MAPPING ID -> INDEX ---
Mapping hoàn tất.


user_index,item_index,weighted_quantity
u32,u32,f64
1041731,7262,1.12782
1010482,16888,0.291545
720193,17947,0.591716
307416,20744,2.884615
928779,17722,1.069519


In [5]:
print("--- BƯỚC 5: TẠO MA TRẬN CSR (SCIPY) ---")

# Chuyển cột Polars sang Numpy Array
rows = df_final["user_index"].to_numpy()  # Dòng là User
cols = df_final["item_index"].to_numpy()  # Cột là Item
data = df_final["weighted_quantity"].to_numpy()

# Kích thước ma trận
n_users = user_map_df.height
n_items = item_map_df.height

# Tạo CSR Matrix
sparse_user_item = sparse.csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

print(f"✅ HOÀN THÀNH! Ma trận sẵn sàng.")
print(f"Shape: {sparse_user_item.shape}")
print(f"Sparsity: {1.0 - (sparse_user_item.nnz / (n_items * n_users)):.6f}")

--- BƯỚC 5: TẠO MA TRẬN CSR (SCIPY) ---
✅ HOÀN THÀNH! Ma trận sẵn sàng.
Shape: (2442305, 20810)
Sparsity: 0.999514


In [6]:
import implicit
import tqdm as tqdm

print("--- BƯỚC 6: HUẤN LUYỆN MODEL ---")

# 1. Khởi tạo và Train TF-IDF
# K là số lượng nearest neighbors để tính similarity, nên set lớn hơn N trong recommend
model_tfidf = implicit.nearest_neighbours.TFIDFRecommender(K=100, num_threads=0)  # num_threads=0 tự động dùng max
model_tfidf.fit(sparse_user_item)  # Dùng User x Item matrix
print("✅ Đã train xong TF-IDF")

# 2. Khởi tạo và Train Cosine (nhanh hơn TF-IDF)
model_cosine = implicit.nearest_neighbours.CosineRecommender(K=100, num_threads=0)
model_cosine.fit(sparse_user_item)  # Dùng User x Item matrix
print("✅ Đã train xong Cosine")

--- BƯỚC 6: HUẤN LUYỆN MODEL ---


c:\ProgramData\anaconda3\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\anaconda3\envs\py312\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.12592864036560059 seconds
  warnings.warn(
100%|██████████| 20810/20810 [00:01<00:00, 18249.41it/s]


✅ Đã train xong TF-IDF


c:\ProgramData\anaconda3\envs\py312\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.1251084804534912 seconds
  warnings.warn(
100%|██████████| 20810/20810 [00:01<00:00, 17660.38it/s]

✅ Đã train xong Cosine


In [7]:
print("--- BƯỚC 7: TẠO CANDIDATE (BATCH PROCESSING) ---")

def generate_candidates(model, model_name, sparse_user_item, user_indices, N=50, batch_size=1000):
    """
    Hàm tạo gợi ý cho danh sách user, trả về Polars DataFrame
    """
    results_user = []
    results_item = []
    results_score = []

    # Lặp qua từng batch user để tiết kiệm RAM
    for start in tqdm.tqdm(range(0, len(user_indices), batch_size), desc=f"Running {model_name}"):
        end = min(start + batch_size, len(user_indices))
        batch_users = user_indices[start:end]

        # Hàm recommend trả về (ids, scores)
        # filter_already_liked_items=False: Vẫn gợi ý món đã mua (để tính việc mua lại)
        ids, scores = model.recommend(batch_users, sparse_user_item[batch_users], N=N, filter_already_liked_items=False)

        # Flatten dữ liệu để đưa vào list
        # ids và scores là mảng 2 chiều, cần flatten ra 1 chiều
        results_user.extend(np.repeat(batch_users, N)) # Lặp lại user_id N lần
        results_item.extend(ids.flatten())
        results_score.extend(scores.flatten())

    # Tạo Polars DataFrame và filter out -1 (items không tồn tại)
    return pl.DataFrame({
        "user_index": results_user,
        "item_index": results_item,
    }).filter(pl.col("item_index") >= 0)

# Lấy danh sách tất cả User Index cần dự đoán
all_user_indices = np.arange(n_users) # n_users lấy từ bước 5

# 1. Chạy model TF-IDF
df_tfidf = generate_candidates(model_tfidf, "tfidf", sparse_user_item, all_user_indices, N=50)

# 2. Chạy model Cosine
df_cosine = generate_candidates(model_cosine, "cosine", sparse_user_item, all_user_indices, N=50)

print("Sample TF-IDF Result:")
df_tfidf.head()

--- BƯỚC 7: TẠO CANDIDATE (BATCH PROCESSING) ---


Running cosine: 100%|██████████| 2443/2443 [01:53<00:00, 21.47it/s]


Sample TF-IDF Result:


user_index,item_index
i32,i32
0,7192
0,18478
0,2566
0,2565
0,9428


In [8]:
print("--- BƯỚC 8: GỘP KẾT QUẢ (CONCAT) + TOP 50 POPULAR ITEMS ---")

# Gộp kết quả từ cả 2 models bằng concat + unique
df_candidates = pl.concat([df_tfidf, df_cosine]).unique(subset=["user_index", "item_index"]).with_columns(
    pl.col("user_index").cast(pl.UInt32),
    pl.col("item_index").cast(pl.UInt32)
)

# Thêm top 50 item phổ biến nhất cho mỗi user (Pure Polars - hiệu suất cao)
top_items = (
    df_grouped
    .group_by("item_id")
    .agg(pl.col("weighted_quantity").sum())
    .sort("weighted_quantity", descending=True).head(50)
    .join(item_map_df, on="item_id").select("item_index")
)
df_popular = user_map_df.select("user_index").join(top_items, how="cross")
df_candidates = pl.concat([df_candidates, df_popular]).unique(subset=["user_index", "item_index"])

print(f"Tổng số dòng Candidate: {df_candidates.height}")

--- BƯỚC 8: GỘP KẾT QUẢ (CONCAT) + TOP 50 POPULAR ITEMS ---
Tổng số dòng Candidate: 244356759


In [9]:
print("--- BƯỚC 9: MAPPING NGƯỢC VỀ ID THẬT ---")

# user_map_df và item_map_df lấy từ BƯỚC 4 ở response trước
# Cấu trúc map df: [user_index, customer_id]

# 1. Join lấy Customer ID
df_final = df_candidates.join(
    user_map_df.with_columns(pl.col("user_index").cast(pl.UInt32)),
    on="user_index",
    how="left"
)

# 2. Join lấy Item ID
df_final = df_final.join(
    item_map_df.with_columns(pl.col("item_index").cast(pl.UInt32)),
    on="item_index",
    how="left"
)

# 3. Chọn cột cần thiết và sắp xếp lại
df_final = df_final.select([
    "customer_id",
    "item_id",
])

print("✅ HOÀN THÀNH TẬP CANDIDATE!")
df_final.head(10)
df_final.write_parquet("train_data/inference_base.parquet")

--- BƯỚC 9: MAPPING NGƯỢC VỀ ID THẬT ---
✅ HOÀN THÀNH TẬP CANDIDATE!
